In [3]:
import pandas as pd
import numpy as np
import time
import inspect

In [20]:
# Helper functions
def timer_function(func=None, n_run=1):

    ti = time.time()
    for n in range(n_run):
        func
    tf = time.time()
    
    print(f'Runtime: {(tf - ti)/n_run} s')

def read_csv_pandas(file_path=None):
    df_data = pd.read_csv(file_path, sep=',', header=None)
    
    return df_data
    

# December 1


In [24]:
timer_function(func=read_csv_pandas("data/test_csv.csv"), n_run=10)
data = read_csv_pandas("data/test_csv.csv")
data

Runtime: 0.0 s


,0,1,2
0,1,a,mdksa sm
1,2,b,jdsasjnj203
2,3,c,dj1992
